# Rote Learning

Die Implementierung des Alpha-Beta-Pruning Algorithmus basiert darauf, für alle möglichen zukünftigen Zustände zu ermitteln, wie wahrscheinlich es ist zu Gewinnen. Je mehr Züge dabei in die Tiefe geschaut werden kann, desto höher ist die Wahrscheinlichkeit, tatsächlich den besten Zug zu machen. Die Laufzeit zur Berechnung der Gewinn-Wahrscheinlichkeit nimmt mit zunehmender Tiefe, aufgrund der sehr schnell ansteigenden Anzahl der Zustände, deutlich zu. Somit ist eine Neuberechnung der Gewinn-Wahrscheinlichkeiten bei jedem Zug nur bis zu einer gewissen Tiefe praktikabel.

An diesem Problem setzt Rote-Learning an, indem es den Alpha-Beta-Pruning Algorithmus um eine elementare Form des Lernens erweitert. Grundlegend werden bei der Verwendung von Rote-Learning alle Zustände, die jemals ausgerechnet wurden, zusammen mit den dazugehörigen errechneten Gewinn-Wahrscheinlichkeit abgespeichert. Anstatt die Wahrscheinlichkeiten dieser Zustände bei jedem Zug neu zu berechnen, können diese nun aus dem Speicher abgerufen werden. Dies spart vor allem bei einer hohen Tiefe einen großen Teil der Rechenzeit. Diese Einsparung der Rechenzeit kann darauf verwendet werden, Zustände, die sich weiter in der Tiefe befinden, zu berechnen. Da die gespeicherten Zustände mit jedem Spiel erweitert werden, verbessert sich das Ergebnis des Algorithmus über Zeit. Es tritt also ein Lern-Effekt ein.

## Implementierung

Um das oben genannte Prinzip von Rote-Learning in Python zu implementieren wurde auf eine Implementierung des Alpha-Beta-Prunings Algorithmus für das Spiel Mühle zurück gegriffen. Dieser wurde im Rahmen der Studienarbeit entwickelt und musste für die Verwendung von Rote-Learning nur noch geringfügig angepasst werden. Um zu evalieren ob Rote-Learning einen Vorteil bringt, werden folgende Notebooks benötigt:

- `nmm-cache` - Implementierung eines persistenten Caches
- `nmm-rote-training` - Implementierung einer Klasse zum trainieren des Caches
- `nmm-alpha-beta-pruning` - Bereits vorhandene Implementierung des Alpha-Beta-Pruning Algorithmus
- `nmm-tournament` - Bereits vorhandene Implementierung zur Evaluierung, welcher Algorithmus besser ist

Desweitern wird noch das `memory_profiler` Packet eingebunden, mit welchem sich die Auslastung des Arbeitsspeichers überwachen lässt.

In [ ]:
from memory_profiler import memory_usage

%run ./nmm-cache.ipynb
%run ./nmm-rote-training.ipynb
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-tournament.ipynb

### Cache

Um den bei Rote-Learning beschriebenen Lern-Effekt zu erzielen und die Zustände zu Speichern wurde eine `Cache` Klasse in dem Notebook `nmm-cache` implementiert. Diese Klasse verfügt über vier Methoden zur Verwaltung des Caches:
- `write` - Speichert einen neuen Zustand im Cache ab
- `read` - Liest die Werte eines Zustandes aus dem Cache aus
- `save` - Speichert den Inhalt des Caches in einer Datei auf dem Datenträger ab
- `load` - Lädt die Daten des Caches aus einer Datei auf dem Datenträger aus
- `clean` - Löscht Einträge, wenn die Anzahl der Einträge `max_size` überschreitet

Um die einzelnen Werte für die Zustände im Cache abzulegen werden sowohl die Zustände als auch die Werte in ein Byte-Array konvertiert. Insgesamt ist ein Eintrag des Caches somit 32 Bytes groß. Um zu verhindern, dass der Cache zu groß wird um ihn im Arbeitsspeicher oder auf dem Datenträger speichern zu können, wurde der Parameter `max_size` eingeführt. Dieser begrenzt den Cache auf eine maximale Anzahl an Einträgen. Um die Performance zu steigern, wird dies jedoch nicht beim jedem Aufruf der `write` Methode geprüft sondern nur beim Aufruf der Methode `clean`. Dies kann dazu führen, dass zwischen den Aufrufen der `clean` Methode mehr Einträge im Cache vorhanden sind, als durch `max_size` spezifiziert. 

In [ ]:
cache = Cache(max_size = 50_000_000)

### Anbindung an Alpha-Beta-Pruning

Da im Alpha-Beta-Pruning Algorithmus bereits die Verwendung eines In-Memrory-Caches vorgesehen ist, mussten nur geringe anpassungen vorgenommen werden. Es musste sichergestellt werden, dass sowohl zum schreiben als auch lesen des Caches die richtigen Methoden aufgerufen werden, sowie dass ein Cache entweder als Parameter übergeben werden kann oder ein neuer instanziiert wird. Nach jeder Berechnung der nächsten besten Züge mithilfe der `bestMoves` Methode wurde der Aufruf der `clean` Methode hinterlegt um die Größe des Caches anzupassen.

###  Traingings-Prozess

Um die Transpositionstabelle zu trainieren, wird die Klasse `Training` aus dem Notebook `nmm-rote-training.ipynb`verwendet. Dazu muss zuerst eine Methode erstellt werden, welche den Algorithmus für die künstliche Intelligenz konfiguriert und generiert. Diese Methode wird `artificial_intelligence_generator` genannt und generiert einen Alpha-Beta-Pruning Algorithmus, welcher den übergebenen Cache und benutzerdefinierte Heuristiken verwendet. Die verwendeten Heuristiken wurden bereits im Rahmen der Studienarbeit ermittelt.

In [ ]:
def artificial_intelligence_generator(cache: Cache):
    customWeights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1)
    return AlphaBetaPruning(cache = cache, weights = customWeights)

Für den Cache wurde sich (wie weiter oben zu sehen) für eine maximale Größe von 50 000 000 Einträgen entschieden. Dies resultiert in eine maximal Größe des Caches von 1,6Gb auf dem Datenträger. Des Weiteren werden die Standardwerte der Training-Klasse verwendet. Das heißt, dass ingesamt 100 Spiele gespielt werden und der Seed nicht angepasst wird. Alle 10 Spiele wird der Cache auf dem Datenträger gespeichert. Dazu wird der Prefix `training-` verwendet.

In [ ]:
training = Training(
    cache = cache, 
    artificial_intelligence = artificial_intelligence_generator,
    path_prefix = 'training-small-'
)

Der Trainings-Prozess wird durch den Aufruf der Methode `train` gestartet und dauert mit der oben genannten konfiguration in etwa 4,5 Stunden. Dabei werden 10 Gb Arbeitsspeicher benötigt.

In [ ]:
# mem_usage = memory_usage(training.train)
# print('Maximum memory usage: %s MB.' % max(mem_usage))

### Trainings Limitierungen
Auch wenn es auf den ersten Blick so erscheint, dass man den Cache unendlich weiter trainieren kann, ist dies mit der aktuellen Implementierung nicht zielführend. Ab einem gewissen Punkt ist das Rekursionslimit der Einträge im Cache so hoch, dass dieses durch weiteres Training nicht erneut erreicht werden kann. Dies ist gut an dem nachfolgenden Beispiel zu sehen. Durch die geringe Größe des Caches sowie dem kleinen Wert für `max_states` kommt man sehr schnell an den Punkt, an dem das Cache-Limit auf 3 erhöht wird. Dadurch werden die meisten Einträge aus dem Cache gelöscht und es verbleiben nur noch `48` im Cache. Dies passiert mehrmals und ist ein Zeichen dafür, dass der Cache nicht mehr trainiert werden kann, da dieser zu klein ist um genügend Einträge zu halten damit man mehr Einträge mit `limit = 3` berechnen kann bevor der `max_states` Wert überschritten wird.

In [ ]:
# t = Training(
#     cache                   = Cache(max_size = 1_000),
#     artificial_intelligence = lambda cache: AlphaBetaPruning(
#         cache      = cache,
#         max_states = 1_000
#     ),
#     path_prefix             = "max-learning-",
#     save_interval           = 1
# )
# t.train()

## Auswertung

Bei der Auswertung wird auf die Klasse `Tournament` aus dem Notebook `nmm-tournament` zurück gegriffen. Diese Klasse besitzt die Methode `play`, welche es ermöglicht verschiedene Algorithmen gegeneinander Antreten zu lassen und die Ergebnisse letzendlich auswertet.

Um zu ermitteln ob die Rote-Learning-Methode einen Vorteil gegenüber einem normalen Cache bietet, tritt ein über 100 Spiele trainierter Cache gegen einen untrainierten Cache an. Dies wird zehn mal mit verschiedenen Seeds wiederholt. Das verändern der Seeds sorgt dabei für einen veränderten Spielverlauf.

Somit sollte ermittelt werden können, ob durch das Training eine Verbesserung der künstlichen Intelligenz eintritt und wenn ja, wie deutlich diese Verbesserung ausfällt.

In [ ]:
for i in range(10):
    Tournament(
        [
            lambda: AlphaBetaPruning(
                name='New Cache',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000)
            ),
            lambda: AlphaBetaPruning(
                name='Trained Cache',
                weights = HeuristicWeights(stones = 3, stash = 3, mills = 2, possible_mills = 1),
                cache = Cache(max_size = 50_000_000, path = 'training-small-final.cache')
            )
        ],
        instances_per_round = 1,
        seed_offset         = i,
        name                = f"small-full-{i}-seed"
    ).play()

Die Auswertung hat ergeben, dass von insgesamt von 20 gespielten Spielen 10 gewonnen wurden. Bei 5 lief es auf ein Unentschieden herraus und 5 wurden verloren. 